In [ ]:
import os
import json
import xarray as xr
import datetime as dt
import rioxarray as rio

In [ ]:
# Sentinel-2 band names
band_names = ["B01", "B02", "B03", "B04", "B05", "B06", "B07", "B08", "B8A", "B09", "B10", "B11", "B12"]

In [ ]:
time_idx = []
datasets = []
for folder, _, filenames in os.walk("output/s2_data"):
    if filenames:
        meta = os.path.join(folder, filenames[0])
        tiff = os.path.join(folder, filenames[1])

        with open(meta) as f:
            data = json.load(f)
        
        from_time = data["request"]["payload"]["input"]["data"][0]["dataFilter"]["timeRange"]["from"]
        ds_name = from_time.split("T")[0]
        
        time = dt.datetime.strptime(ds_name, '%Y-%m-%d')
        
        ds = rio.open_rasterio(tiff)
        ds.name = ds_name
        ds["band"] = band_names
        ds.rio.write_crs("epsg:4326", inplace=True)
        
        # ds = ds.rio.reproject("EPSG:3857")

        time_idx.append(time)
        datasets.append(ds)

In [ ]:
time_idx = sorted(time_idx)

In [ ]:
output_ds = xr.concat(datasets, dim=xr.Variable('time', sorted(time_idx)))
output_ds.name = "values"
output_ds["time"] = output_ds.indexes['time'].normalize()

In [ ]:
output_ds.to_netcdf("output/s2_data.nc")